###### Kernel notebook for getting airbnb housing reviews sentiment

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/ML/Houses/'
data_reviews_csv = pd.read_csv(path + 'reviews.csv')

In [7]:
#reviews unnecessary columns and join comments of the same listing_id
class ReviewCleaner(TransformerMixin):

    def transform(self, X, **transform_params):
        str_comments = X['comments'].astype(str)
        X = pd.DataFrame(X).assign(comments = str_comments)
        X = X.groupby('listing_id')['comments'].apply(' '.join).reset_index()

        vader_analyzer = SentimentIntensityAnalyzer()
        X['sentiment'] = pd.Series(np.random.randn(len(X['comments'])), index=X.index)
        X['sentiment'] = X['comments'].apply(lambda x: vader_analyzer.polarity_scores(x))
        return X

    def fit(self, X, y=None, **fit_params):
        return self
    
reviews_cleaner = Pipeline([
    ('clean reviews', ReviewCleaner())
])

In [13]:
data_reviews_raw = data_reviews_csv.copy()
data_reviews = reviews_cleaner.transform(data_reviews_raw)

In [14]:
data_reviews.to_csv(path + 'sentimented_reviews.csv', index = False)